In [ ]:
#pip install plotly==4.1.0

import plotly.express as px
import plotly.graph_objs as go

import pandas as pd
import numpy as np
from plyfile import PlyData, PlyElement

import math
from math import sqrt


In [ ]:
from plotly.subplots import make_subplots
from plotly import tools

clasificar_j = pd.read_csv('testawards.csv', delimiter = ',') ## Import the dataset desde sklearn.datasets

plydata = PlyData.read("topo_octubre_hpfin.ply")

#for element in plydata.elements:
    #print (element)

nr_points=plydata.elements[0].count
nr_faces=plydata.elements[1].count

points=np.array([plydata['vertex'][k] for k in range(nr_points)])
#points[0]

xa,yb,zc=zip(*points)

In [ ]:
from ipywidgets import widgets
from IPython.display import clear_output

t = np.linspace(-1,1.2,2000)
x = clasificar_j['SigmaS3']
y = clasificar_j['SigmaS1']


def SetColor(x):
    if(x == 0):
        return 'rgb(0, 200, 200)'
    elif(x == 1):
        return "red"
    
def NameUG(x):
    if(x == 0):
        return "Used Data"
    elif(x == 1):
        return "Discarded"
    
def UCSUG(x):
    if(x == 0):
        return "Used Sigma One"
    elif(x == 1):
        return "Discarded Sigma One"

s3j = [c*0.1 - 10 for c in range(400)]
s3mogi = [c for c in range(100)]
s1mogi = [a * 3.4 for a in s3mogi]
    
def Hoek(x):
    if(x=="TODOLIX"):
        b=10
        a=24
        s1j = [(n + a * (sqrt(b * (n / a) + 1))) for n in s3j if (b * (n / a) + 1)>=0]
        s3p = [(n) for n in s3j if (b * (n / a) + 1)>=0]
    if(x=="PRIM"):
        b=18
        a=49
        s1j = [(n + a * (sqrt(b * (n / a) + 1))) for n in s3j if (b * (n / a) + 1)>=0]
        s3p = [(n) for n in s3j if (b * (n / a) + 1)>=0]
    if(x=="SEC"):
        b=18
        a=49
        s1j = [(n + a * (sqrt(b * (n / a) + 1))) for n in s3j if (b * (n / a) + 1)>=0]
        s3p = [(n) for n in s3j if (b * (n / a) + 1)>=0]
    if(x=="PRIMSEC"):
        b=9
        a=33
        s1j = [(n + a * (sqrt(b * (n / a) + 1))) for n in s3j if (b * (n / a) + 1)>=0]
        s3p = [(n) for n in s3j if (b * (n / a) + 1)>=0]
    return s3p, s1j, a, b

## definicion del grafico mediante plotly, la variable a cambiar corresponde a "hue"

def plotpro(Select):

    fig = make_subplots(rows=2, cols=3, specs=[[{"type": "bar"},None, {"type": "scatter"}],
                                               [None, None, {'is_3d': True}]], column_widths=[0.2, 0.1, 0.7], row_heights=[1, 1])

  
    for UG, stress in clasificar_j.groupby(Select):
        fig.add_trace(go.Scatter(x=stress.SigmaS3, y=stress.SigmaS1, name=NameUG(UG), mode='markers',
                                 marker=dict(size=10, opacity=0.7, line=dict(width=1), color=SetColor(UG))), row=1, col=3)
        fig.update_xaxes(title_text="Minor Principal Stress (Mpa)",titlefont=dict(family='Rockwell',size=15,color='rgb(17, 60, 160)'), range=[-10, 30], row=1, col=3)
        fig.update_yaxes(title_text="Major Principal Stress (Mpa)",titlefont=dict(family='Rockwell',size=15,color='rgb(17, 60, 160)'), range=[0, 350], row=1, col=3)

        fig.add_trace(go.Histogram(y=stress.SigmaS1, name= UCSUG(UG), xaxis='x2', yaxis='y2', opacity=0.6, marker_color=SetColor(UG)), row=1, col=1)

        # Agregamos los puntos en las coordenadas del rajo
        fig.add_trace(
    go.Scatter3d(x=clasificar_j['x'], y=clasificar_j['y'], z=clasificar_j['z'], marker=dict(color='rgb(0, 200, 200)'), mode='markers', name='Used', showlegend=False),
    row=2, col=3)  
        fig.add_trace(go.Mesh3d(x=xa, y=yb, z=zc, color='#8c564b', opacity=0.50, name='Topography'), row=2, col=3)
        fig.update_layout(scene_aspectmode='manual',scene_aspectratio=dict(x=1, y=1, z=0.2))
        
    # agregamos ajuste por Hoek
    fig.append_trace(go.Scatter(x=Hoek(Select)[0], y=Hoek(Select)[1], name='Hoek Fit: Sci=%.0f' % Hoek(Select)[2]+ ' , mi=%.0f' % Hoek(Select)[3], mode='lines', line = dict(color='firebrick', width=4, dash='dot')), 1, 3)
    fig.append_trace(go.Scatter(x=s3mogi, y=s1mogi, name='Mogi Line', mode='lines', line = dict(color='gray', width=2, dash='dot')), 1, 3)    
    fig.update_xaxes(title_text="Histogram Sigma One (Mpa)",titlefont=dict(family='Rockwell',size=15,color='rgb(17, 60, 160)'), range=[30, 0], mirror=True, row=1, col=1)
    fig.update_yaxes(range=[0, 350], showticklabels=False, row=1, col=1)
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
    fig.layout.update(title_text="Curve Fitting for Generalized Hoek-Brown",width = 900, height=900)

    fig.layout.update(
        yaxis3 = dict(scaleanchor = "y3", scaleratio = 1, range=[0.5, 3], domain=[0, 0.45], showgrid=False,showticklabels=False, mirror=True),plot_bgcolor='rgba(228, 222, 249, 0.65)',
        xaxis3 = dict(scaleanchor = "x3", scaleratio = 1, range=[0, 2.25], domain=[0.45, 0.9], showgrid=False, showticklabels=False, mirror=True))


    fig.show()
  

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
#interact(plotpro,description='Test:', Select={'Lixiviation':"TODOLIX", 'Primary Mineralization':"PRIM", 'Secondary Mineralization':"SEC"})

plotpro("SEC")